In [1]:
import sys
import torch
import tntorch as tn
import numpy as np
sys.path.append('..')
import lib

rho = np.array([[1., 0.], [0., 0.]]) # target state
dim = rho.shape[0]

projectors_cnt = 10
measurements_cnt = 100

train_size = projectors_cnt * measurements_cnt
train_X, train_y = lib.generate_dataset(rho, projectors_cnt, measurements_cnt)
train_y = train_y.astype('float64')

sigma = lib.simulator.randomMixedState(rho.shape[0])
tensor_rank = 2*dim
sigma_real, sigma_imag = [tn.Tensor(x, ranks_tt=tensor_rank, requires_grad=True) for x in [np.real(sigma), np.imag(sigma)]]

epoches = 10

def trace(tensor):
    if len(tensor.shape) == 2:
        return sum([tensor[i,i] for i in range(tensor.shape[0])])
    if len(tensor.shape) == 3:
        return sum([tensor[:, i,i] for i in range(tensor.shape[1])])

def loss(sigma_real, sigma_imag):
    res = 0
    for E_m,y_m in zip(train_X, train_y.astype('float64')):
        E_real, E_imag = [tn.Tensor(x) for x in [np.real(E_m), np.imag(E_m)]]
        res += ((trace(E_real*sigma_real-E_imag*sigma_imag)-y_m)**2)
    return res
    
print('Loss before: %f' %sum((trace(train_X*sigma)-train_y)**2))

tn.optimize([sigma_real, sigma_imag], loss)

sigma = sigma_real.torch().detach().cpu().numpy() + 1j*sigma_imag.torch().detach().cpu().numpy()
sigma /= trace(sigma)
print('Loss after: %f' %sum((trace(train_X*sigma)-train_y)**2))

/home/artem/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:37: ComplexWarning: Casting complex values to real discards the imaginary part


Loss before: 0.271475
iter: 0       | loss:   0.271475 | total time:    0.0164
iter: 350     | loss:   0.014092 | total time:   11.7112 <- converged (tol=0.0001)
Loss after: 0.014546


/home/artem/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:43: ComplexWarning: Casting complex values to real discards the imaginary part
